<a href="https://colab.research.google.com/github/Junkicch/GatoPretoVsCaramelo/blob/main/GatoPretoXCaramelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
(train_ds, val_ds), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

In [ ]:
IMG_SIZE = 160

def format_image(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_ds = train_ds.map(format_image).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(format_image).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

In [ ]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")

model = tf.keras.Sequential([
    base_model,
    global_avg,
    output_layer
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

In [ ]:
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

In [ ]:
loss, acc = model.evaluate(val_ds)
print(f"Val Loss: {loss:.4f} | Val Accuracy: {acc:.4f}")

In [ ]:
acc_all = history.history['accuracy'] + history_fine.history['accuracy']
val_acc_all = history.history['val_accuracy'] + history_fine.history['val_accuracy']

plt.plot(acc_all, label="Treino")
plt.plot(val_acc_all, label="Validação")
plt.xlabel("Épocas")
plt.ylabel("Acurácia")
plt.legend()
plt.show()

In [ ]:



img_path = "./image/2000px.webp"

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))

img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)

if pred[0][0] > 0.5:
    print(f"É um cachorro 🐶 (confiança {pred[0][0]*100:.2f}%)")
else:
    print(f"É um gato 🐱 (confiança {(1-pred[0][0])*100:.2f}%)")




img = mpimg.imread(img_path)
plt.imshow(img)
plt.axis("off")
plt.title("Cachorro 🐶" if pred[0][0] > 0.5 else "Gato 🐱")
plt.show()
